In [1]:
import cv2
import numpy as np
import json

In [2]:
image = None
current_points = []
parking_slots = []
draw_mode_active = False

In [3]:
def save_parking_slots_to_json(parking_slots, output_file):
    parking_data = [
        {
            "slot_id": slot_id,
            "coordinates": coordinates
        }
        for coordinates, slot_id in parking_slots
    ]

    with open(output_file, "w") as f:
        json.dump(parking_data, f, indent=4)

In [4]:
def is_point_in_parking_slot(point, polygon):
    result = cv2.pointPolygonTest(np.array(polygon, dtype=np.int32), point, False)
    return result >= 0  

In [5]:
def mouse_callback(event, x, y, flags, param):
    global draw_mode_active, current_points, parking_slots, image

    if not draw_mode_active:
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        if len(current_points) < 4:
            current_points.append((x, y))
            draw_interface()
            if len(current_points) > 1:
                cv2.line(image, current_points[-2], current_points[-1], (0, 255, 0), 6)
                draw_interface()

            if len(current_points) == 4:
                cv2.line(image, current_points[-1], current_points[0], (0, 255, 0), 6)
                parking_slot_id = len(parking_slots) + 1
                parking_slots.append((current_points.copy(), parking_slot_id))
                current_points.clear()
                draw_mode_active=False
                draw_interface()

In [6]:
def draw_interface():
    global draw_mode_active, image
    img_copy = image.copy()

    for slot, slot_id in parking_slots:
        cv2.polylines(img_copy, [np.array(slot)], True, (0, 255, 0), 4)
        cv2.putText(img_copy, f"ID {slot_id}", tuple(slot[0]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)

    mode_text = "DRAW MODE: ON" if draw_mode_active else "DRAW MODE: OFF"
    mode_color = (0, 255, 0) if draw_mode_active else (0, 0, 255)
    cv2.putText(img_copy, mode_text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, mode_color, 6)

    h, w, _ = img_copy.shape
    legend_height = 250

    cv2.rectangle(img_copy, (0, h - legend_height), (w, h), (0, 0, 0), -1)

    cv2.putText(img_copy, "MENU:", (20, h - 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)

    if draw_mode_active:
        cv2.putText(img_copy, "Left-click: Define Park Slot (4 Coordinates)", (20, h - 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        cv2.putText(img_copy, "ESC: Abort Draw Mode", (20, h - 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
    else:
        cv2.putText(img_copy, "D: Enter Draw Mode", (20, h - 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        cv2.putText(img_copy, "S: Save | Z: Undo Last Park Slot | Q: Quit", (20, h - 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
        
    cv2.imshow("Define Parking Slots", img_copy)

In [ ]:
def define_parking_slots(image_path, output_file="parking_slots.json"):
    global draw_mode_active, image, parking_slots

    original_image = cv2.imread(image_path)
    image = original_image.copy()
    cv2.imshow("Define Parking Slots", image)
    draw_interface()
    cv2.setMouseCallback("Define Parking Slots", mouse_callback)

    while True:
        key = cv2.waitKey(1) & 0xFF
        draw_interface()

        if key == ord('q'):  
            break

        elif key == ord('z') and parking_slots:
            parking_slots.pop()
            image = original_image.copy()
            draw_interface()

        elif key == ord('d'):
            draw_mode_active = True
            draw_interface()

        elif key == 27: # ESC
            if draw_mode_active:
                current_points.clear()
                image = original_image.copy()
                draw_mode_active = False
                draw_interface()

        elif key == ord('s'): # S
            save_parking_slots_to_json(parking_slots, output_file)
            print(f"Parking slots saved to {output_file}.")
            break

    cv2.destroyAllWindows()
    return parking_slots, image

In [ ]:
if __name__ == "__main__":
    define_parking_slots("Testaufnahme_RGB.png")